## <center>Pleasantly Parallel</center>
### <center> Linh B. Ngo </center>
### <center> CPSC 3620 </center>

- Embarrassingly parallel/naturally parallel/pleasantly parallel
- “A computation that can obviously be divided into a number of completely different parts, each of which can be executed by a separate process.”


- No communication or very little communication among the processes.
- Each process can do its tasks without any interaction with the other processes.


**Python's NumPy library**
- Supports n-dimensional array
- Provides optimized and vectorized operations on multi-dimensional arrays in large-scale computations

**mpi4py and NumPy**
- lower-case communitation routines (send, recv, gather, scatter, reduce) in mpi4py
operate on Python's original data types with no optimization
- upper-case communication routines (Send, Recv, Bcast, Scatter, Gather, Reduce ...) require
Numpy and have syntax similar to the original MPI_xxxx routines in C

**Upper-case point-to-point: MPI.Send**

Comm.Send(buf, dest = 0, tag = 0)

Parameters:	
- Comm (MPI comm) – communicator we wish to query
- buf (choice) – data to send
- dest (integer) – rank of destination
- tag (integer) – message tag

**Upper-case point-to-point: MPI.Recv**

Comm.Recv(buf, source = 0, tag = 0, Status status = None)

Parameters:	
- Comm (MPI comm) – communicator we wish to query
- buf (choice) – initial address of receive buffer
- source (integer) – rank of source
- tag (integer) – message tag
- status (Status) – status of object

In [1]:
%%writefile codes/mpi4py/sendrecvUpper.py
#!/usr/bin/env python
# sendrecvUpper.py
import numpy
from mpi4py import MPI
comm = MPI.COMM_WORLD
rank = comm.Get_rank(); print(rank)
a = numpy.zeros((4), dtype=numpy.int)
status = MPI.Status()
print (a)
if rank == 0:
    a = numpy.array([1,2,4])
    print (a)
    comm.Send(a, dest=1, tag = 1000)
if rank == 1:
    comm.Recv(a, source = 0, tag = MPI.ANY_TAG, status = status)
    print (status.Get_source())
    print (status.Get_tag())
    print (a)

Overwriting codes/mpi4py/sendrecvUpper.py


In [2]:
!module load gcc/5.3.0 openmpi/1.10.3; mpirun -np 2 codes/mpi4py/sendrecvUpper.py

0
[0 0 0 0]
[1 2 4]
1
[0 0 0 0]
0
1000
[1 2 4 0]


**Uppercase collective: MPI.Bcast**

Comm.Bcast(buf, root=0)

Parameters:	
- Comm (MPI comm) – communicator across which to broadcast
- buf (choice) – buffer
- root (int) – rank of root operation

In [3]:
%%writefile codes/mpi4py/bcastUpper.py
#!/usr/bin/env python
# bcastUpper.py
import numpy
from mpi4py import MPI
comm = MPI.COMM_WORLD
rank = comm.Get_rank()
rand_num = numpy.zeros(1)
print (rand_num[0])
if rank == 0:
    rand_num[0] = numpy.random.uniform(0)
    print(rand_num[0])
comm.Bcast(rand_num, root = 0)
print ("Process", rank, "has the number", rand_num[0])

Overwriting codes/mpi4py/bcastUpper.py


In [11]:
!module load gcc/5.3.0 openmpi/1.10.3; mpirun -np 4 codes/mpi4py/bcastUpper.py

0.0
Process 1 has the number 0.786986419305
0.0
Process 2 has the number 0.786986419305
0.0
Process 3 has the number 0.786986419305
0.0
0.786986419305
Process 0 has the number 0.786986419305


**Uppercase collective: MPI.Scatter**

Comm.Scatter(sendbuf, recvbuf, root)

Parameters:	
- sendbuf (choice) – address of send buffer (significant only at root)
- recvbuf (choice) – address of receive buffer
- root (int) – rank of sending process

In [12]:
%%writefile codes/mpi4py/scatterUpper.py
#!/usr/bin/env python
# scatterUpper.py
import numpy
from mpi4py import MPI
comm = MPI.COMM_WORLD
rank = comm.Get_rank();size = comm.Get_size();LENGTH = 3
if rank == 0:
    x = numpy.linspace(1,size*LENGTH,size*LENGTH)
    print ("Original data: ", x)
else:
    x = None
x_local = numpy.zeros(LENGTH)
comm.Scatter(x, x_local, root=0)
#you should notice that only the root process has a value for x that
#is not "None"
print ("process", rank, "x:", x)
print ("process", rank, "x_local:", x_local)

Overwriting codes/mpi4py/scatterUpper.py


In [13]:
!module load gcc/5.3.0 openmpi/1.10.3; mpirun -np 4 codes/mpi4py/scatterUpper.py

Original data:  [  1.   2.   3.   4.   5.   6.   7.   8.   9.  10.  11.  12.]
process 1 x: None
process 2 x: None
process 3 x: None
process 0 x: [  1.   2.   3.   4.   5.   6.   7.   8.   9.  10.  11.  12.]
process 0 x_local: [ 1.  2.  3.]
process 1 x_local: [ 4.  5.  6.]
process 2 x_local: [ 7.  8.  9.]
process 3 x_local: [ 10.  11.  12.]


**Uppercase collective: MPI.Gather**

Comm.Gather(sendbuf, recvbuf, root)

Parameters:	
- sendbuf (choice) – address of send buffer (significant only at root)
- recvbuf (choice) – address of receive buffer
- root (int) – rank of receiving process

In [14]:
%%writefile codes/mpi4py/gatherUpper.py
#!/usr/bin/env python
# gatherUpper.py
import numpy
from mpi4py import MPI
comm = MPI.COMM_WORLD
rank = comm.Get_rank()
size = comm.Get_size()
LENGTH = 3
x = None
x_local = numpy.linspace(rank*LENGTH,(rank+1)*LENGTH, LENGTH)
print(x_local)
if rank == 0:
    x = numpy.zeros(size*LENGTH)
    print (x)
comm.Gather(x_local, x, root=0)

#you should notice that only the root process has a value for x that
#is not "None"
print ("process", rank, "x:", x)
print ("process", rank, "x_local:", x_local)

Overwriting codes/mpi4py/gatherUpper.py


In [15]:
!module load gcc/5.3.0 openmpi/1.10.3; mpirun -np 4 codes/mpi4py/gatherUpper.py

[ 0.   1.5  3. ]
[ 3.   4.5  6. ]
process 1 x: None
process 1 x_local: [ 3.   4.5  6. ]
[ 6.   7.5  9. ]
process 2 x: None
[  9.   10.5  12. ]
process 3 x: None
process 3 x_local: [  9.   10.5  12. ]
[ 0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.]
process 0 x: [  0.    1.5   3.    3.    4.5   6.    6.    7.5   9.    9.   10.5  12. ]
process 0 x_local: [ 0.   1.5  3. ]
process 2 x_local: [ 6.   7.5  9. ]


**Uppercase collective: MPI.Reduce**

Comm.Reduce(sendbuf, recvbuf, Op op = MPI.SUM, root = 0)

Parameters:	
- Comm (MPI comm) – communicator we wish to query
- sendbuf (choice) – address of send buffer
- recvbuf (choice) – address of receive buffer (only significant at root)
- op (handle) – reduce operation
- root (int) – rank of root operation

- MPI.MAX:	maximum
- MPI.MIN:	minimum
- MPI.SUM:	sum
- MPI.PROD:	product
- MPI.LAND:	logical and
- MPI.BAND:	bit-wise and
- MPI.LOR:	logical or
- MPI.BOR:	bit-wise or
- MPI.LXOR:	logical xor
- MPI.BXOR:	bit-wise xor
- MPI.MAXLOC:	max value and location
- MPI.MINLOC:	min value and location

In [18]:
%%writefile codes/mpi4py/reduceUpper.py
#!/usr/bin/env python
# reduceUpper.py
import numpy
from mpi4py import MPI
comm = MPI.COMM_WORLD
rank = comm.Get_rank()
rankF = numpy.array(float(rank))
total = numpy.zeros(1)
comm.Reduce(rankF,total, op=MPI.MAX, root = 0)
print (rank, total)

Overwriting codes/mpi4py/reduceUpper.py


In [19]:
!module load gcc/5.3.0 openmpi/1.10.3; mpirun -np 4 codes/mpi4py/reduceUpper.py

0 [ 3.]
1 [ 0.]
2 [ 0.]
3 [ 0.]


In [23]:
%%writefile ./codes/mpi4py/reduceUpper2.py
#!/usr/bin/env python
# reducerUpper2.py
import numpy
from mpi4py import MPI
comm = MPI.COMM_WORLD
rank = comm.Get_rank()
LENGTH = 3
x = None
x_local = numpy.linspace(rank*LENGTH,(rank+1)*LENGTH, LENGTH)
print (rank, x_local)
total = numpy.zeros(LENGTH)
comm.Reduce(x_local,total, op=MPI.SUM, root = 0)
print (rank, total)

Overwriting ./codes/mpi4py/reduceUpper2.py


In [24]:
!module load gcc/5.3.0 openmpi/1.10.3; mpirun -np 4 codes/mpi4py/reduceUpper2.py

1 [ 3.   4.5  6. ]
1 [ 0.  0.  0.]
2 [ 6.   7.5  9. ]
2 [ 0.  0.  0.]
3 [  9.   10.5  12. ]
3 [ 0.  0.  0.]
0 [ 0.   1.5  3. ]
0 [ 18.  24.  30.]


#### <center> Example: Trapezoid Calculation

<center> 
    <img src="pictures/06/trapezoid01.png" width="400"/>
</center>

In [25]:
N = 8; a = 0; b = 2; h = (b - a)/N;

In [26]:
# With 4 processors (cores)
size = 4; rank = 1
local_N = N / size
local_a = a + rank * h * local_N
local_b = local_a + h * local_N
print (local_a, local_b)

0.5 1.0


- Which workload goes to which process?
```
if (rank == i) {
	do great things
}
```
- Start with small number of processes
- Calculation workload assignment manually for each count of processes
- Generalize assignment for process i based on sample calculations


In [27]:
%%writefile codes/mpi4py/trapezoid1.py
#!/usr/bin/env python
# trapezoid1.py
import numpy
from mpi4py import MPI
comm = MPI.COMM_WORLD
rank = comm.Get_rank(); size = comm.Get_size()
N = 1000; a = 0; b = 1; h = (b - a)/N
def f(x):
    return x*x
local_N = N / size
local_a = a + rank * local_N * h
partial_result = numpy.zeros(1)
sum = numpy.zeros(1)
for i in range(0,int(local_N)):
    partial_result += (f(local_a) + f(local_a + h)) * h / 2
    local_a = local_a + h
comm.Reduce(partial_result,sum, op=MPI.SUM, root=0)
if rank == 0:
    print ("The integral is %s" % (sum))

Overwriting codes/mpi4py/trapezoid1.py


In [28]:
!module load gcc/5.3.0 openmpi/1.10.3; mpirun -np 4 codes/mpi4py/trapezoid1.py

The integral is [ 0.3333335]


#### <center> Does each process receive the same amount of work?

In [29]:
%%writefile codes/mpi4py/trapezoid2.py
#!/usr/bin/env python
# trapezoid2.py
import numpy
from mpi4py import MPI
comm = MPI.COMM_WORLD
rank = comm.Get_rank(); size = comm.Get_size()
N = 8; a = 0; b = 1; h = (b - a)/N
def f(x):
    return x*x
local_N = N / size
local_a = a + rank * local_N * h
partial_result = numpy.zeros(1)
sum = numpy.zeros(1)
for i in range(0,int(local_N)):
    partial_result = partial_result + (f(local_a) + f(local_a + h)) * h / 2
    local_a = local_a + h
comm.Reduce(partial_result,sum, op=MPI.SUM, root=0)
print ("Process ", rank, " has ", partial_result[0])
if rank == 0:
    print ("The integral is %s" % (sum[0]))

Overwriting codes/mpi4py/trapezoid2.py


In [30]:
!module load gcc/5.3.0 openmpi/1.10.3; mpirun -np 4 codes/mpi4py/trapezoid2.py

Process  1  has  0.037109375
Process  2  has  0.099609375
Process  3  has  0.193359375
Process  0  has  0.005859375
The integral is 0.3359375


In [31]:
%%writefile codes/mpi4py/newEvenodd.py
#!/usr/bin/env python
# evenodd.py
from mpi4py import MPI
comm = MPI.COMM_WORLD
rank = comm.Get_rank()
size = comm.Get_size()
name = MPI.Get_processor_name()
if (rank % 2 == 0):
    print ("Process %s is even" % (rank))
else:
    print ("Process %s is odd" % (rank))

Writing codes/mpi4py/newEvenodd.py


In [37]:
!module load gcc/5.3.0 openmpi/1.10.3; chmod 755 codes/mpi4py/newEvenodd.py; mpirun -np 2 codes/mpi4py/newEvenodd.py

Process 0 is even
Process 1 is odd


<center> 
    <img src="pictures/06/static-wa.png" width="400"/>
</center>

<center> 
    <img src="pictures/06/cyclic-wl.png" width="400"/>
</center>

In [38]:
%%writefile codes/mpi4py/trapezoidCyclic.py
#!/usr/bin/env python
# trapezoidCyclic.py
import numpy; from mpi4py import MPI
comm = MPI.COMM_WORLD
rank = comm.Get_rank(); size = comm.Get_size()
N = 3000; a = 0; b = 1; h = (b - a)/N
def f(x):
    return x*x
local_N = N / size; local_a = a + rank * h
partial_result = numpy.zeros(1)
sum = numpy.zeros(1)
for i in range(0,int(local_N)):
    partial_result = partial_result + (f(local_a) + f(local_a + h)) * h / 2
    local_a = local_a + size * h
comm.Reduce(partial_result,sum, op=MPI.SUM, root=0)
print ("Process ", rank, " has ", partial_result[0])
if rank == 0:
    print ("The integral is %s" % (sum[0]))

Overwriting codes/mpi4py/trapezoidCyclic.py


In [39]:
!module load gcc/5.3.0 openmpi/1.10.3; mpirun -np 4 codes/mpi4py/trapezoidCyclic.py

Process  1  has  0.0832916435185
Process  2  has  0.0833749768519
Process  3  has  0.0834583657407
Process  0  has  0.0832083657407
The integral is 0.333333351852


<center> 
    <img src="pictures/06/dynamic-wl.png" width="800"/>
</center>

In [ ]:
%%writefile codes/mpi4py/trapezoidDynamic.py
#!/usr/bin/env python
# trapezoidDynamic.py
from mpi4py import MPI
import numpy
comm = MPI.COMM_WORLD
rank = comm.Get_rank(); size = comm.Get_size()
def f(x):
    return x*x
partial_result = numpy.zeros(1)
local_result = numpy.zeros(1)
sum = numpy.zeros(1)
h = numpy.zeros(1) 
a = numpy.zeros(1)
status = MPI.Status()

if (rank == 0):
    N = 1000; a[0]= 0; b = 1; h[0]= (b - a)/N;
    comm.Bcast(h, root = 0)
    count = 0;
    for i in range(1, size):
        comm.Send(a,dest = i,tag = 1)
        count = count + 1
else:
    comm.Bcast(h, root = 0)
    comm.Recv(a,source = 0, tag = 1,status = status)

if (rank != 0):
    while True:
        local_result[0] = h[0] * (f(a[0]) +  f(a[0] + h[0])) / 2
        partial_result[0] += local_result[0];
        comm.Send(local_result,dest = 0,tag = 1)      
        comm.Recv(a, source = 0, tag = MPI.ANY_TAG, status = status)
        if status.Get_tag() != 1:
            break
            
if (rank == 0):
    while (count < N):
        comm.Recv(partial_result, source = MPI.ANY_SOURCE, tag = MPI.ANY_TAG, status = status)
        sum[0] = sum[0] + partial_result[0]
        local_a = numpy.array([a + count * h])
        comm.Send(local_a, dest = status.Get_source(), tag = 1)
        count = count + 1      
    for i in range (1,size):
        comm.Recv(partial_result, source = MPI.ANY_SOURCE, tag = MPI.ANY_TAG, status = status)
        sum = sum + partial_result
    for i in range (1,size):
        comm.Send(a,dest = i,tag = 0)
    print ("The integral is %s" % (sum))
else:
    print ("The partial_integral is %s" % (partial_result))

In [40]:
!module load gcc/5.3.0 openmpi/1.10.3; mpirun -np 4 codes/mpi4py/trapezoidDynamic.py

The integral is [ 0.33333349]
The partial_integral is [ 0.11077794]
The partial_integral is [ 0.11144461]
The partial_integral is [ 0.11111094]
